In [1]:
#!pip install arrow icalendar

In [2]:
from nuliga_event_search import *
SEASON = 'Sommer 2023'
CLUBS = {
    'TC Alfeld Mannschaften': 'https://tnb.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubTeams?club=16473',
    'TSV Gronau Mannschaften': 'https://tnb.liga.nu/cgi-bin/WebObjects/nuLigaTENDE.woa/wa/clubTeams?club=16486',
}

process_season_clubs(season=SEASON, clubs=CLUBS, output_dir=Path('.'))

could not find expected nr. of tables > 3
TC Alfeld II: Herren Verbandsliga Gr. 207


,Tag,Datum,Heimmannschaft,Gastmannschaft,Matchpunkte,Sätze,Spiele,Spielbericht
0,So.,07.05.2023,TC Alfeld II,TC GW Gifhorn,,,,offen
1,So.,14.05.2023,MTV Wolfenbüttel,TC Alfeld II,,,,offen
2,Sa.,03.06.2023 15:00,TC Alfeld II,TC Sommerbostel II,,,,offen
3,So.,18.06.2023,Tennisclub Gieboldehausen e.V.,TC Alfeld II,,,,offen
4,So.,02.07.2023,TC Alfeld II,SV GW Waggum,,,,offen
5,So.,20.08.2023,TV Salzgitter-Gebhardshagen,TC Alfeld II,,,,offen


[Errno 2] No such file or directory: 'TC Alfeld Mannschaften\\Sommer 2023 TC Alfeld II Herren Verbandsliga Gr. 207.ics'
TC Alfeld: Herren 30 Verbandsklasse Gr. 306


,Tag,Datum,Heimmannschaft,Gastmannschaft,Matchpunkte,Sätze,Spiele,Spielbericht
0,So.,14.05.2023,TV Vienenburg,TC Alfeld,,,,offen
1,So.,04.06.2023,MTV Markoldendorf,TC Alfeld,,,,offen
2,So.,18.06.2023,TV Salzgitter-Gebhardshagen,TC Alfeld,,,,offen
3,So.,25.06.2023,TC Alfeld,TC PTB e.V. Braunschweig,,,,offen
4,So.,02.07.2023,TC Alfeld,Heidberger TC III,,,,offen
5,So.,20.08.2023,TC Alfeld,Hildesheimer TV,,,,offen


[Errno 2] No such file or directory: 'TC Alfeld Mannschaften\\Sommer 2023 TC Alfeld Herren 30 Verbandsklasse Gr. 306.ics'
TC Alfeld: Herren 40 Bezirksklasse Gr. 402


...

[Errno 2] No such file or directory: 'TSV Gronau Mannschaften\\Sommer 2023 TSV Gronau Damen 30 Bezirksklasse Gr. 077.ics'
TSV Gronau: Junioren B Regionsklasse (2er) Gr. 812


,Tag,Datum,Heimmannschaft,Gastmannschaft,Matchpunkte,Sätze,Spiele,Spielbericht
0,Sa.,06.05.2023,TSV Gronau,SC Barienrode,,,,offen
1,Sa.,13.05.2023,SV Eintracht Bad Salzdetfurth,TSV Gronau,,,,offen
2,Sa.,03.06.2023,TSV Gronau,TC Alfeld,,,,offen
3,Sa.,17.06.2023,TSV Gronau,MTV Almstedt,,,,offen
4,Sa.,01.07.2023,MTV Bodenburg,TSV Gronau,,,,offen


[Errno 2] No such file or directory: 'TSV Gronau Mannschaften\\Sommer 2023 TSV Gronau Junioren B Regionsklasse (2er) Gr. 812.ics'
TSV Gronau: Juniorinnen A Regionsklasse (2er) Gr. 940


,Tag,Datum,Heimmannschaft,Gastmannschaft,Matchpunkte,Sätze,Spiele,Spielbericht
0,Sa.,06.05.2023,TSV Gronau,TC Holzminden,,,,offen
1,Sa.,13.05.2023,MTV Harsum,TSV Gronau,,,,offen
2,Sa.,03.06.2023,TC Aerzen II,TSV Gronau,,,,offen
3,Sa.,17.06.2023,TSV Gronau,SV Eintracht Bad Salzdetfurth II,,,,offen
4,Sa.,01.07.2023,TSV Gronau,Hildesheimer TC Rot-Weiß II,,,,offen


[Errno 2] No such file or directory: 'TSV Gronau Mannschaften\\Sommer 2023 TSV Gronau Juniorinnen A Regionsklasse (2er) Gr. 940.ics'


In [3]:
#!rm *.ics